# [Advent of Code 2020 Day 12](https://adventofcode.com/2020/day/12#part2)

Oh boy, directional questions. I always have a bittersweet relationship with these because they're so hard to visualize (despite being... visual), but they're so satisfying once you actually get it.

## Initial setup

In [1]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing

In [2]:
def parse_input(filename: str) -> Any:

    gen = yield_line(filename)

    result = []

    for line in gen:
        result.append(parse(r"([NSEWLRF])(.*)", line))

    result = [(direction, int(magnitude)) for direction, magnitude in result]

    return result

## Part 1
Just doing things the straightforward way. Though with some other quirks:
- Modular arithmetic so I can branchlessly handle turning
- Coordinates used to store direction and magnitude

In [3]:
def part_one(data: list[tuple[str, int]]) -> int | str:

    directions = (
        (0, 1), (1, 0), (0, -1), (-1, 0)
    )

    east, south, west, north = directions

    absolute_direction = {
        "N": north, "S": south, "E": east, "W": west,
    }

    dir_index = 0
    x, y = 0, 0

    def turn(degrees: int) -> None:
        nonlocal dir_index
        assert degrees in [90, 180, 270, 360]
        dir_index = (dir_index + (degrees // 90)) % 4

    for direction, magnitude in data:
        if direction in "NSEW":
            dx, dy = absolute_direction[direction]
            for _ in range(magnitude):
                x += dx
                y += dy
        elif direction == "L":
            turn(360 - magnitude)
        elif direction == "R":
            turn(magnitude)
        elif direction == "F":
            dx, dy = directions[dir_index]
            for _ in range(magnitude):
                x += dx
                y += dy
        else:
            raise Exception(f"Invalid direction {direction}")

    return abs(x) + abs(y)

In [4]:
%%ipytest
def test_part_one():
    assert part_one([("F", 10)]) == 10
    assert part_one([("F", 10), ("N", 3)]) == 13
    assert part_one([("F", 10), ("N", 3), ("F", 7)]) == 20
    assert part_one([("F", 10), ("N", 3), ("F", 7), ("R", 90)]) == 20
    assert part_one([("F", 10), ("N", 3), ("F", 7), ("R", 90), ("F", 11)]) == 25
    assert part_one(parse_input("example1")) == 25
    assert part_one(parse_input("input")) == 2270

.                                                                                            [100%]
1 passed in 0.01s


## Part 2
Had to whip out my Desmos for this. The only differences were:
  - Waypoint exists as a custom directional bearing; fortunately I've been using this method of displacement since part one, so it was a trivial change
  - "F" had to be changed

In [5]:
def part_two(data: list[tuple[str, int]]) -> int | str:

    directions = (
        (0, 1), (1, 0), (0, -1), (-1, 0)
    )

    east, south, west, north = directions

    absolute_direction = {
        "N": north, "S": south, "E": east, "W": west,
    }

    x, y = 0, 0
    waypoint_x, waypoint_y = -1, 10

    def turn(degrees: int) -> None:
        nonlocal waypoint_x, waypoint_y
        assert degrees in [90, 180, 270, 360]
        for _ in range(degrees // 90):
            waypoint_x, waypoint_y = waypoint_y, -waypoint_x

    for direction, magnitude in data:
        if direction in "NSEW":
            dx, dy = absolute_direction[direction]
            for _ in range(magnitude):
                waypoint_x += dx
                waypoint_y += dy
        elif direction == "L":
            turn(360 - magnitude)
        elif direction == "R":
            turn(magnitude)
        elif direction == "F":
            dx, dy = waypoint_x, waypoint_y
            for _ in range(magnitude):
                x += dx
                y += dy
        else:
            raise Exception(f"Invalid direction {direction}")

    return abs(x) + abs(y)

In [6]:
%%ipytest
def test_part_two():
    assert part_two(parse_input("example1")) == 286
    assert part_two(parse_input("input")) == 138669

.                                                                                            [100%]
1 passed in 0.01s
